In [46]:
import torch
import torch.nn as nn
import torch.nn.functional as f

In [25]:
class neural_net(nn.Module):
    
    def __init__(self):
        super(neural_net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = f.max_pool2d(self.conv1(x), (2, 2))
        x = f.max_pool2d(self.conv2(x), (2, 2))
        x = x.view(-1, self.num_flat_features(x))
        x = f.relu(self.fc1(x))
        x = f.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [26]:
net = neural_net()
print(net)

neural_net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [27]:
params = list(net.parameters())
print(len(params))
print(params[0].size())

10
torch.Size([6, 1, 5, 5])


In [31]:
inp = torch.randn(1, 1, 32, 32)
out = net(inp)
print(out)

tensor([[-0.0054,  0.0631, -0.1276, -0.0416,  0.0509,  0.0025,  0.0234,
         -0.0086, -0.0177, -0.0876]])


In [29]:
net.zero_grad()
out.backward(torch.randn(1, 10))

In [37]:
output = net(inp)
target = torch.arange(1, 11)
target = target.view(1, -1)
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(38.7231)


In [39]:
net.zero_grad()
print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([ 0.,  0.,  0.,  0.,  0.,  0.])
conv1.bias.grad after backward
tensor([-0.0926,  0.1832,  0.1048,  0.0273,  0.0635,  0.0072])


In [44]:
learning_rate = 0.01
for _ in net.parameters():
    _.data.sub_(_.grad.data * learning_rate)

In [47]:
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr = 0.01)
optimizer.zero_grad()
output = net(inp)
loss = criterion(output, target)
loss.backward()
optimizer.step()